# Challenge: Múltiples variables almacenadas en una columna.

Vamos a tratar ahora con el siguiente dataset: Registros de tuberculosis de la Organización Mundial de la Salud
Este conjunto de datos documenta el recuento de casos confirmados de tuberculosis por país, año, edad y sexo.

## Problemas:
- Algunas columnas contienen varios valores: sexo y edad.
- Mezcla de ceros y valores perdidos NaN. Esto se debe al proceso de recopilación de datos y la distinción es importante para este conjunto de datos.

In [32]:
import pandas as pd

In [33]:
tb = pd.read_csv("tb-raw.csv")
tb

,country,year,m014,m1524,m2534,m3544,m4554,m5564,m65,mu,f014
0,AD,2000,0.0,0.0,1.0,0.0,0,0,0.0,NaN,NaN
1,AE,2000,2.0,4.0,4.0,6.0,5,12,10.0,NaN,3.0
2,AF,2000,52.0,228.0,183.0,149.0,129,94,80.0,NaN,93.0
3,AG,2000,0.0,0.0,0.0,0.0,0,0,1.0,NaN,1.0
4,AL,2000,2.0,19.0,21.0,14.0,24,19,16.0,NaN,3.0
5,AM,2000,2.0,152.0,130.0,131.0,63,26,21.0,NaN,1.0
6,AN,2000,0.0,0.0,1.0,2.0,0,0,0.0,NaN,0.0
7,AO,2000,186.0,999.0,1003.0,912.0,482,312,194.0,NaN,247.0
8,AR,2000,97.0,278.0,594.0,402.0,419,368,330.0,NaN,121.0
9,AS,2000,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN


In [34]:
tb.drop(columns=["mu"],inplace=True)
tb


,country,year,m014,m1524,m2534,m3544,m4554,m5564,m65,f014
0,AD,2000,0.0,0.0,1.0,0.0,0,0,0.0,NaN
1,AE,2000,2.0,4.0,4.0,6.0,5,12,10.0,3.0
2,AF,2000,52.0,228.0,183.0,149.0,129,94,80.0,93.0
3,AG,2000,0.0,0.0,0.0,0.0,0,0,1.0,1.0
4,AL,2000,2.0,19.0,21.0,14.0,24,19,16.0,3.0
5,AM,2000,2.0,152.0,130.0,131.0,63,26,21.0,1.0
6,AN,2000,0.0,0.0,1.0,2.0,0,0,0.0,0.0
7,AO,2000,186.0,999.0,1003.0,912.0,482,312,194.0,247.0
8,AR,2000,97.0,278.0,594.0,402.0,419,368,330.0,121.0
9,AS,2000,NaN,NaN,NaN,NaN,1,1,NaN,NaN


Para ordenar este conjunto de datos, necesitamos eliminar los diferentes valores del encabezado y SEPARARLOS en filas.

### Instrucciones:
- Primero, necesitaremos fusionar las columnas de sexo + grupo de edad en una sola.
- Una vez que tengamos esa única columna, derivaremos tres columnas de ella: sexo, age_lower y age_upper. Con ellos, podremos construir correctamente un conjunto de datos ordenado.

In [35]:
# Utiliza melt como en primer challenge para realizar el melt sobre las columnas que correspondan
tb1=pd.melt(frame=tb, id_vars=["country","year"], var_name="sexo + grupo de edad", value_name="cases")
tb1

,country,year,sexo + grupo de edad,cases
0,AD,2000,m014,0.0
1,AE,2000,m014,2.0
2,AF,2000,m014,52.0
3,AG,2000,m014,0.0
4,AL,2000,m014,2.0
...,...,...,...,...
75,AM,2000,f014,1.0
76,AN,2000,f014,0.0
77,AO,2000,f014,247.0
78,AR,2000,f014,121.0


In [36]:
tb1Expanded=tb1["sexo + grupo de edad"].str.split("(\d+)", expand=True)
tb1Expanded.drop(columns=[2],inplace=True)
tb1Expanded.columns=["sexo","grupo de edad"]
tb1Expanded

,sexo,grupo de edad
0,m,014
1,m,014
2,m,014
3,m,014
4,m,014
...,...,...
75,f,014
76,f,014
77,f,014
78,f,014


In [37]:
tb1=tb1.drop(columns=["sexo + grupo de edad"])
tb2=pd.concat([tb1, tb1Expanded ],axis=1)
tb2

,country,year,cases,sexo,grupo de edad
0,AD,2000,0.0,m,014
1,AE,2000,2.0,m,014
2,AF,2000,52.0,m,014
3,AG,2000,0.0,m,014
4,AL,2000,2.0,m,014
...,...,...,...,...,...
75,AM,2000,1.0,f,014
76,AN,2000,0.0,f,014
77,AO,2000,247.0,f,014
78,AR,2000,121.0,f,014


In [63]:
tb2_age=tb2["grupo de edad"].str.split("", expand=True)
tb2_age.drop(columns=[0,5],inplace=True)
tb2_age["Age"]=0
tb2_age

,1,2,3,4,Age
0,0,1,4,,0
1,0,1,4,,0
2,0,1,4,,0
3,0,1,4,,0
4,0,1,4,,0
...,...,...,...,...,...
75,0,1,4,,0
76,0,1,4,,0
77,0,1,4,,0
78,0,1,4,,0


In [72]:
for i in range(80):
    if  tb2["grupo de edad"][i]=='65':
        tb2_age["Age"][i]="65"
    elif tb2["grupo de edad"][i]=='014':
        tb2_age["Age"][i]=("0-14")
    else:
        tb2_age["Age"][i]=(tb2_age[1][i]+tb2_age[2][i])+"-"+(tb2_age[3][i]+tb2_age[4][i])
       
tb2_age


,1,2,3,4,Age
0,0,1,4,,0-14
1,0,1,4,,0-14
2,0,1,4,,0-14
3,0,1,4,,0-14
4,0,1,4,,0-14
...,...,...,...,...,...
75,0,1,4,,0-14
76,0,1,4,,0-14
77,0,1,4,,0-14
78,0,1,4,,0-14


In [90]:
tb_f=pd.concat([tb2, tb2_age["Age"]],axis=1)
tb_f.drop(columns=["grupo de edad"],inplace=True)
tb_f1=tb_f[tb_f["cases"].notna()]
tb_f1

,country,year,cases,sexo,Age
0,AD,2000,0.0,m,0-14
1,AE,2000,2.0,m,0-14
2,AF,2000,52.0,m,0-14
3,AG,2000,0.0,m,0-14
4,AL,2000,2.0,m,0-14
...,...,...,...,...,...
74,AL,2000,3.0,f,0-14
75,AM,2000,1.0,f,0-14
76,AN,2000,0.0,f,0-14
77,AO,2000,247.0,f,0-14


In [91]:
tb_f1.sort_values(["country","sexo","Age"],ascending=True)

,country,year,cases,sexo,Age
0,AD,2000,0.0,m,0-14
10,AD,2000,0.0,m,15-24
20,AD,2000,1.0,m,25-34
30,AD,2000,0.0,m,35-44
40,AD,2000,0.0,m,45-54
...,...,...,...,...,...
48,AR,2000,419.0,m,45-54
58,AR,2000,368.0,m,55-64
68,AR,2000,330.0,m,65
49,AS,2000,1.0,m,45-54


## Resultado
Al finalizar, deberías obtener un dataframe similar a este:

![image.png](https://storage.googleapis.com/campus-cvs/lectures/tidyDataChallenge2.PNG)